In [180]:
import gym
import numpy as np

In [181]:
env = gym.make('CartPole-v0')

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [182]:
n_games = 10000
learning_rate = 0.01
weight_decay = 0.01
gamma = 0.95
epsilon = 0.01
max_t = 100

In [198]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

class Quality_Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.input = nn.Linear(5, 5)
        self.output = nn.Linear(5, 1)
    def forward(self, state, action):
        x = torch.cat([state, action])
        x = F.relu(self.input(x))
        x = F.softmax(self.output(x))
        return x

class Quality():
    def __init__(self):
        self.net = Quality_Net()
        self.optimizer = optim.Adam(self.net.parameters(), lr=learning_rate, weight_decay=weight_decay)
        self.lossFunction = nn.MSELoss()
    def eval(self, state, action):
        return self.net(Variable(torch.FloatTensor(state)), Variable(torch.FloatTensor([action])))
    def train(self, state, action, quality):
        self.net.train()
        self.optimizer.zero_grad()
        predicted_quality = self.eval(state, action)
        #quality = quality.detach()
        quality = Variable(torch.FloatTensor([quality]))
        quality.requires_grad = False
        loss = self.lossFunction(predicted_quality, quality)
        loss.backward()
        self.optimizer.step()
        return loss.data[0]
    
quality_function = Quality()

In [199]:
def policy(state):
    if np.random.uniform(0, 1) < epsilon:
        return np.random.choice(env.action_space.n)
    else:
        best_action = 0
        best_quality = quality_function.eval(state, 0).data[0]
        for action in range(env.action_space.n):
            quality = quality_function.eval(state, action)
            if quality.data[0] > best_quality:
                best_quality = quality.data[0]
                best_action = action
        return best_action

def value(state):
    best_action = 0
    best_quality = quality_function.eval(state, 0).data[0]
    for action in range(env.action_space.n):
        quality = quality_function.eval(state, action)
        if quality.data[0] > best_quality:
            best_quality = quality.data[0]
            best_action = action
    return best_quality

In [201]:
for i in range(n_games):
    state = env.reset()
    total_reward = 0
    for t in range(max_t):
        action = policy(state)
        old_state = state
        state, reward, done, _ = env.step(action)
        env.render()
        total_reward += reward
        #quality_function.train(old_state, action, (1-lr) * quality_function.eval(old_state, action) + lr * (reward + value(state)))
        quality_function.train(old_state, action, reward + value(state))
        if (done): break
    if i%100 == 0:
        print("Iteration", i, "Reward", total_reward)

/Users/mattzhang/py3_kernel/lib/python3.5/site-packages/ipykernel/__main__.py:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Iteration 0 Reward 11.0
Iteration 100 Reward 8.0
Iteration 200 Reward 9.0
Iteration 300 Reward 10.0
Iteration 400 Reward 10.0
Iteration 500 Reward 10.0
Iteration 600 Reward 10.0
Iteration 700 Reward 10.0
Iteration 800 Reward 9.0
Iteration 900 Reward 9.0
Iteration 1000 Reward 8.0
Iteration 1100 Reward 9.0
Iteration 1200 Reward 8.0
Iteration 1300 Reward 9.0
Iteration 1400 Reward 9.0
Iteration 1500 Reward 9.0
Iteration 1600 Reward 8.0
Iteration 1700 Reward 9.0
Iteration 1800 Reward 10.0
Iteration 1900 Reward 10.0
Iteration 2000 Reward 11.0
Iteration 2100 Reward 9.0
Iteration 2200 Reward 10.0
Iteration 2300 Reward 10.0
Iteration 2400 Reward 9.0
Iteration 2500 Reward 8.0
Iteration 2600 Reward 9.0
Iteration 2700 Reward 10.0
Iteration 2800 Reward 10.0
Iteration 2900 Reward 10.0
Iteration 3000 Reward 10.0
Iteration 3100 Reward 10.0
Iteration 3200 Reward 8.0
Iteration 3300 Reward 10.0
Iteration 3400 Reward 11.0
Iteration 3500 Reward 9.0
Iteration 3600 Reward 10.0
Iteration 3700 Reward 9.0
Itera